<a href="https://colab.research.google.com/github/Syed-Ahmed-Omer/CNN_G/blob/main/CNN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import random
import shutil
import cv2
import glob
import zipfile
import io

#Data uploading

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
zf = zipfile.ZipFile(io.BytesIO(upload['data.zip']), 'r') #extacting the zip file
zf.extractall()

In [ ]:
cd /content/data

/content/data


#Data Preprocessing

In [ ]:
#create the directories and folders
directory = '/content/data'
if os.path.isdir('/content/data/train') is False:
  os.makedirs('/content/data/train/Glaco')
  os.makedirs('/content/data/train/N_Glaco')
  os.makedirs('/content/data/test/Glaco')
  os.makedirs('/content/data/test/N_Glaco')

In [ ]:
#moving the images in train and test folders.
for i in random.sample(glob.glob('/content/data/1*'), 30):
  shutil.move(i, '/content/data/train/N_Glaco')
for i in random.sample(glob.glob('/content/data/1*'), 10):
  shutil.move(i, '/content/data/test/N_Glaco')
for i in random.sample(glob.glob('/content/data/2*'), 30):
  shutil.move(i, '/content/data/train/Glaco')
for i in random.sample(glob.glob('/content/data/2*'), 10):
  shutil.move(i, '/content/data/test/Glaco')

#Data Augmentation

In [ ]:
train_path = '/content/data/train'
test_path = '/content/data/test'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255, #normalization
        shear_range=0.2, #data aug.
        zoom_range=0.2, #data aug.
        horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=1,
        classes = ['Glaco', 'N_Glaco'])

Found 60 images belonging to 2 classes.


In [ ]:
#Shuffle = False because to create a confusion matriz at end.
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=1,
        classes = ['Glaco', 'N_Glaco'],
        shuffle = False)

Found 20 images belonging to 2 classes.


#CNN Architecture

In [ ]:
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Building the model architecture
model = Sequential()

model.add(Conv2D(32, 3, 3, activation= 'relu', input_shape =(150,150,3)))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64, 3, 3, activation= 'relu'))
model.add(MaxPool2D(2,2))

model.add(Flatten())

model.add(Dense(100, activation= 'relu'))

model.add(Dense(2, activation= 'softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
y_real = test_generator.labels #assigning labels to y_real

In [ ]:
model.fit(train_generator,
          epochs = 20,
          steps_per_epoch = 60,
          )

Epoch 1/20
60/60 [==============================] - 2s 23ms/step - loss: 0.6697 - accuracy: 0.5458
Epoch 2/20
60/60 [==============================] - 1s 23ms/step - loss: 0.3758 - accuracy: 0.8469
Epoch 3/20
60/60 [==============================] - 1s 23ms/step - loss: 0.2167 - accuracy: 0.9396
Epoch 4/20
60/60 [==============================] - 1s 22ms/step - loss: 0.2279 - accuracy: 0.8618
Epoch 5/20
60/60 [==============================] - 1s 22ms/step - loss: 0.0654 - accuracy: 0.9919
Epoch 6/20
60/60 [==============================] - 1s 23ms/step - loss: 0.0305 - accuracy: 0.9899
Epoch 7/20
60/60 [==============================] - 1s 22ms/step - loss: 0.0880 - accuracy: 0.9401
Epoch 8/20
60/60 [==============================] - 1s 22ms/step - loss: 0.0469 - accuracy: 1.0000
Epoch 9/20
60/60 [==============================] - 1s 22ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 10/20
60/60 [==============================] - 1s 22ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 11/

#Model Predictions

In [ ]:
y_pred = np.round(model.predict(test_generator))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
y_real

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

In [ ]:
y_pred = np.argmax(y_pred, axis = -1)

In [ ]:
test_generator.class_indices

{'Glaco': 0, 'N_Glaco': 1}

In [ ]:
cm = confusion_matrix(y_real, y_pred)


In [ ]:
cm

array([[10,  0],
       [ 0, 10]])

#OBSERVATION

In [ ]:
#observation
'''
#As I increase CNN layers depth above 3rd layer, model was not learning anything got stuck at local minima
 loss and accuracy decresed and fluctated and was abt 50% only. 
#so, had to take only 2 layers of CNN which gave very good accuracy.
#Adding or removing another dense layer of 50 or 100 units did not make any difference so stickked with only one layer.'''

##saving the model

In [ ]:
cd /content/data

/content/data


In [ ]:
model.save('CNN_G.h5')

#Testing on a new image

In [ ]:
#doing all the necessary preprocessing and loading an image and checking the model on that image

In [ ]:
path = '/content/data/CNN_G.h5'

In [ ]:
model = load_model('CNN_G.h5')

In [ ]:
import cv2

In [ ]:
image = files.upload() #uploading a healthy retinal image

Saving healthy.jpg to healthy.jpg


In [ ]:
img_arr = cv2.imread('/content/data/healthy.jpg') #converting img to array using open cv

In [ ]:
img_arr = cv2.resize(img_arr, (150,150)) # resizing the imported image

In [ ]:
img_arr = img_arr / 255.0 #normalizing the pixles

In [ ]:
img_arr.shape

(150, 150, 3)

In [ ]:
img_arr = img_arr.reshape(1, 150,150,3) #reshaping as req. by the model

In [ ]:
prediction = model.predict(img_arr) #prediction of the img

In [ ]:
np.round(prediction)

array([[0., 1.]], dtype=float32)

In [ ]:
test_generator.class_indices #knowing the indices of the labels

{'Glaco': 0, 'N_Glaco': 1}

In [ ]:
np.argmax(prediction) #model correctly predicts the image as healthy with no galcoma.

1

In [ ]:
#similary testing unseen image og Glaucoma on the model 

In [ ]:
image = files.upload()

Saving G1.jpg to G1.jpg


In [ ]:
img_arr = cv2.imread('/content/data/G1.jpg')
img_arr = cv2.resize(img_arr, (150,150))
img_arr = img_arr / 255.0
img_arr = img_arr.reshape(1, 150,150,3)
prediction = model.predict(img_arr)



In [ ]:
np.argmax(prediction) #model correctly predicts the image as Glaucoma

0

#Result

In [ ]:
#Models does a great job classifying the images.